In [ ]:
from datetime import datetime
import json
import requests

def extract_sensor_data(sensor_id: str) -> dict:
    start = datetime(2020, 9, 9).timestamp()
    end = datetime(2020, 9, 16).timestamp()
    fields = "pm2.5_atm"

    url = f"https://api.purpleair.com/v1/sensors/{sensor_id}/history"
    api_key = "7CD6D264-0A25-11EE-BD21-42010A800008"
    headers = {"X-API-Key":api_key}
    payload = {
        "start_timestamp":start,
        "end_timestamp":end, 
        "average":30,
        "fields":fields,        
            }

    r = requests.get(url, headers=headers, params=payload)
    data = r.json()

    return data

orange_day_data = []
count = 0
with open("data/sensors.csv", "r") as f:
    f.readline()
    for line in f.readlines():
        print(count)

        sensor_id = line.split(",")[0]
        historical_data = extract_sensor_data(sensor_id)
        orange_day_data.append(historical_data)

        count += 1


print(orange_day_data)

with open("data/orange_day_data_testing_9_16_pm25.json", "w") as f:
    json.dump(orange_day_data, f, ensure_ascii=False, indent=2)
        

In [ ]:
import json
import pandas as pd
from datetime import timedelta

with open("data/orange_day_data_testing_9_16_pm25.json", "r") as f:
    historical_data = json.load(f)

historical_data_list = []
historical_data_fields = ["sensor_index","time_stamp", "pm2.5_atm"]
for sensor_data in historical_data:
    if "error" in sensor_data:
        continue
    elif len(sensor_data["data"]) == 0: # skip if there is no data
        continue

    sensor_index = sensor_data["sensor_index"]

    for time_slice in sensor_data["data"]:
        time_slice.insert(0, sensor_index)
        historical_data_list.append(time_slice)

df = pd.DataFrame(historical_data_list, columns=historical_data_fields)
df.time_stamp = df.time_stamp.apply(
        lambda x: datetime(1970, 1, 1) + timedelta(seconds=x) - timedelta(hours=4)
    )
df.to_csv("data/orange_day_data_testing_9_16_pm25.csv", index=False)

In [27]:
import pandas as pd
from datetime import datetime

df = pd.read_csv("data/orange_day_data_testing_9_16_pm25.csv")
sensors = pd.read_csv("data/sensors.csv")
df = df.sort_values('time_stamp')
df = df.join(sensors.set_index("sensor_index"), on="sensor_index")
average_aq_df = df[(df["location_type"] == 0) &
                   (df["sensor_index"] != 21227) &
                   (df["sensor_index"] != 65713) &
                   (df["sensor_index"] != 64697) &
                   (df["sensor_index"] != 1742) &
                   (df["sensor_index"] != 19727) & 
                   (df["sensor_index"] != 37715) &
                   (df["sensor_index"] != 65985)].groupby("time_stamp")["pm2.5_atm"].mean()
average_aq_df.to_csv("data/orange_day_average_pm25.csv")

pd.set_option('display.max_rows', None)
df = df[df["sensor_index"] != 21227].sort_values('pm2.5_atm', ascending=False)
df[df["time_stamp"] == "2020-09-09 00:00:00"].head(100)

,sensor_index,time_stamp,pm2.5_atm,last_modified,date_created,last_seen,private,name,icon,location_type,...,latitude,longitude,altitude,channel_state,channel_flags,channel_flags_manual,channel_flags_auto,confidence,confidence_auto,confidence_manual
14348,23941,2020-09-09 00:00:00,109.3240,1636317476,1546297604,1691797928,0,sf_outersunset_inside_am_v2,0,1,...,37.760900,-122.497500,153,3,0,0,0,100,100.0,100.0
35667,55927,2020-09-09 00:00:00,47.2455,1663785887,1589922790,1691797884,0,office,0,1,...,37.763040,-122.395930,96,3,0,0,0,100,100.0,100.0
34628,55609,2020-09-09 00:00:00,23.8645,1649874052,1589574589,1691216279,0,622 Spruce St,0,0,...,37.783320,-122.453230,225,3,0,0,0,100,100.0,100.0
49610,63991,2020-09-09 00:00:00,23.6405,1600034142,1598973867,1691797848,0,350 Sanchez,0,0,...,37.763440,-122.431240,112,3,0,0,0,100,100.0,100.0
35245,55807,2020-09-09 00:00:00,17.4840,1598561422,1589832940,1691797833,0,Mendell &amp; La Salle Ave,0,0,...,37.736557,-122.388310,88,3,0,0,0,100,100.0,100.0
39650,61155,2020-09-09 00:00:00,16.8050,1599112792,1597093322,1691797850,0,Dolores and 21st (Outdoor),0,0,...,37.756798,-122.425606,186,3,0,0,0,28,28.0,28.0
44341,62233,2020-09-09 00:00:00,16.3615,1598908041,1598243295,1691797868,0,19th &amp; Noe,0,0,...,37.759460,-122.432660,170,3,0,0,0,100,100.0,100.0
5019,19433,2020-09-09 00:00:00,16.1710,1598137024,1542414465,1691797900,0,Duboce Triangle,0,0,...,37.767307,-122.435570,236,3,0,0,0,100,100.0,100.0
26518,44055,2020-09-09 00:00:00,15.4225,1598158976,1576519379,1691797847,0,15th and Castro St.,0,0,...,37.766277,-122.435684,217,3,0,0,0,100,100.0,100.0
31571,54177,2020-09-09 00:00:00,14.9970,1598213128,1588282376,1691797905,0,Roosevelt Way,0,0,...,37.762783,-122.444340,430,3,0,0,0,100,100.0,100.0


In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)

df = pd.read_csv("data/orange_day_data_testing_9_16_pm25.csv")
df = df.drop(columns=["sensor_index"])
df = df.groupby(['time_stamp'])["pm2.5_atm"].mean()
df.head(15)
# df = df.rename(columns={"pm2.5_atm":"mean_pm2.5_atm"})
# df.to_csv("data/orange_day_average_pm25.csv")
